<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Astroquery Beta Test - April 2025</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

_This notebook focuses specifically on recent developments related to the transition from the WDB backend to the TAP backend._ - [DFS-19635](https://jira.eso.org/browse/DFS-19635)

Feedback expected from beta testers:

 - What absolutely needs to be fixed before the release
 - What would be nice to be fixed before the release
 - Feature ideas and improvements for subsequent releases

The current astroquery ESO module documentation will mostly work: https://astroquery.readthedocs.io/en/latest/eso/eso.html
For the parts that unevitably needed changes, the updated documentation is found in the `docs` folder in this repository:
`./docs/AstroqueryEsoDocumentation-Beta.pdf` and `./docs/AstroqueryEsoDocstrings-Beta.pdf`


In [ ]:
!bash install_astroquery.sh

## Handy imports and functions

In [ ]:
import os
import pandas as pd
import astropy.table.table as aqtable
from pyvo.dal.exceptions import DALQueryError, DALServiceError
from IPython.display import clear_output

def table_to_csv(aqtable: aqtable, filepath: str):
    aqtable.to_pandas().to_csv(filepath)
    print(f"Table saved to {filepath}")

## ESO instance

In [ ]:
from astroquery.eso import Eso

eso_instance = Eso()

eso_instance.maxrec = 1000

print(f"ESO tap_url = {eso_instance._tap_url()}")

## Query Raw Data

In [ ]:
my_table = eso_instance.query_main()
table_to_csv(my_table, "./eso_aq_example_query_main.csv")
# my_table[:2].show_in_notebook()

## Query Phase 3 Data

In [ ]:
available_surveys = eso_instance.list_surveys()

print(pd.DataFrame({"survey": available_surveys}).to_string())

In [ ]:
my_table = eso_instance.query_surveys()

table_to_csv(my_table, "./eso_aq_example_query_surveys.csv")
# my_table[:2].show_in_notebook()

In [ ]:
my_table = eso_instance.query_surveys('SPHERE')

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere.csv")
# my_table.show_in_notebook()

In [ ]:
my_table = eso_instance.query_surveys(['SPHERE', 'VEGAS'])
my_table_2 = eso_instance.query_surveys('SPHERE, VEGAS')
assert all(my_table.values_equal(my_table_2))

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere_vegas.csv")

#my_table.show_in_notebook()

## Query Instrument Specific Data

In [ ]:
available_instruments = eso_instance.list_instruments()

if available_instruments is None:
    print("No available instruments: This is expected, since the ISTs are only available in the INT server for the moment.")

    print("Available instruments in the INT server:\n")

    os.environ["ESO_TAP_URL"] = "http://dfidev5.hq.eso.org:8123/tap_obs"
    available_instruments = eso_instance.list_instruments()
    del os.environ["ESO_TAP_URL"]

print(pd.DataFrame({"instrument": available_instruments}).to_string())

In [ ]:
try:
    my_table = eso_instance.query_instrument("midi")
except DALQueryError as e:
    print(f"DALQueryError expected since the ISTS are available only in the INT server.")
    os.environ["ESO_TAP_URL"] = "http://dfidev5.hq.eso.org:8123/tap_obs"
    print("Querying now the INT server...")
    my_table = eso_instance.query_instrument("midi")
finally:
    del(os.environ["ESO_TAP_URL"])

table_to_csv(my_table, "./eso_aq_example_query_instrument_midi.csv")
# my_table.show_in_notebook()

## Query some columns

In [ ]:
my_table = eso_instance.query_main(columns=["target", "instrument", "date_obs", "access_url", "datalink_url"])
table_to_csv(my_table, "./eso_aq_example_query_main_columns.csv")
#my_table[0:3].show_in_notebook()

## Info before launching a query

### Rows and columns

In [ ]:
eso_instance.query_main(help=True) # TO IMPROVE

# Originally: query_something(help=True)
# But `help` is a reserved word in python

# IDEAS:
# eso_instance.query_main(print_info=True)
# help(eso_instance.query_main) eso.query_main?

### Number of records

In [ ]:
q_str = eso_instance.query_main(count_only=True)
count_m = eso_instance.query_main(instrument="MUSE", count_only=True)
#print(f"Counts: {count_n:0.2e}, {count_m:0.2e}")

q_str = eso_instance.query_surveys('MUSE', count_only=True)
count_m = eso_instance.query_surveys('MUSE', ra=18, dec=0 , radius=1, count_only=True)
#print(f"Counts: {count_n:0.2e}, {count_m:0.2e}")

In [ ]:
survey_nrows = {"survey": [], "num_records": []}
for x in available_surveys:
    n = eso_instance.query_surveys(x, count_only=True)
    survey_nrows["num_records"].append(n)
    survey_nrows["survey"].append(x)
    print(x, n)

clear_output()
print(pd.DataFrame(survey_nrows).to_string())

### Column names as list

In [ ]:
column_list = list(eso_instance.query_main(top=1).columns)
print(pd.DataFrame(column_list).to_string())

## Limit the number of records - _top_ vs _maxrec_

 - `top`: "I don't care how many records there are, give me only the _top N_, because my science requires only those."
 - `maxrec`: "Give me as many records as possible given my computational resources and time constraints. Please warn me if results are truncated."

Key Differences:

 - Scope: `TOP` is used within the ADQL query to limit the number of rows returned by that specific query. In contrast, `maxrec` is a TAP service parameter that sets an upper limit on the number of records any query can return, acting as a server-side `safeguard` control.

 - Implementation: `TOP` is specified in the query language (ADQL), whereas `maxrec` is a parameter of the TAP service protocol, often set outside the query itself.

In [ ]:
eso_instance.maxrec = None 
eso_instance.query_main(top=3)

In [ ]:
eso_instance.maxrec = 3
eso_instance.query_main()

In [ ]:
eso_instance.maxrec = 5
eso_instance.query_main(top=10)

In [ ]:
eso_instance.maxrec = 10
eso_instance.query_main(top=5)

In [ ]:
eso_instance.maxrec = 1000

## Filter in advance

In [ ]:
my_table = eso_instance.query_surveys("sphere, vegas",
                                          columns="obs_survey, calib_level, multi_ob, filter, s_pixel_scale, instrument_name" ,
                                          calib_level=3,
                                          multi_ob='M')

table_to_csv(my_table, "./eso_aq_example_filter_vegas.csv")

## Filter afterwards

In [ ]:
my_table[my_table["s_pixel_scale"] > 0.2]

## Free ADQL query

In [ ]:
eso_instance.query_tap_service("Select obs_survey, count(*) from ivoa.ObsCore group by obs_survey order by obs_survey")

In [ ]:
eso_instance.query_tap_service("Select obs_survey, calib_level, multi_ob, filter, s_pixel_scale, instrument_name "
                               "from ivoa.ObsCore where "
                               "obs_survey in ('sphere', 'vegas') and "
                               "calib_level=3 and "
                               "multi_ob='M' and "
                               "s_pixel_scale < 0.2")

## Bug fixing and possible improvements

### Authenticated queries - MISSING

 - Even when the login is correct, the query is unauthenticated.
 - No authenticated queries possible --> No risk of data leaks.

In [ ]:
eso_instance.login(username='jcarmonaops')

In [ ]:
h = eso_instance._get_auth_header()
eso_instance._session.headers = {**eso_instance._session.headers,
                                 **eso_instance._get_auth_header()}

tap_auth = pyvo.dal.TAPService(eso_instance._tap_url(), session=eso_instance._session)
tap = pyvo.dal.TAPService(eso_instance._tap_url())

print(eso_instance._session.headers)

_ = tap.search("select count(*) from dbo.raw")
print(_) # 34424443

try:
    _ = tap_auth.search("select count(*) from dbo.raw")
    print(_) # 48186661
except DALServiceError as e:
    print("Error in auth query")


In [ ]:
{**{1:2}, **{1:3}}

### Help message - IMPROVEMENT

In [ ]:
eso_instance.query_surveys(print_help=True)
# help(query_...)
# make the output look nicer

### Cone search - BUG - SOLVED

In [ ]:
q_str = eso_instance.query_main(asdasd='asdasd', query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(ra=18, dec=0 , radius=0.1, query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(ra=18, dec=0 , radius=0.1, asdasd='asdasd', query_str_only=True)
print(q_str)

In [ ]:
my_table = eso_instance.query_surveys(surveys='MUSE', ra=150, dec=-59, radius=1, columns="target_name, s_ra, s_dec, s_region")
my_table = eso_instance.query_surveys(ra=150, dec=-59, radius=1, columns="target_name, s_ra, s_dec, s_region")
#table_to_csv(my_table, "./eso_aq_example_cone_search.csv")

### `count_only` - BUG - Related to cone search - Solved

In [ ]:
# Filtering gives the same count ... ?

# - Qery survey #

count_m = eso_instance.query_surveys('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print(count_m)
print()

count_m = eso_instance.query_surveys(ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print(count_m)
print()

# - Query main - #

count_m = eso_instance.query_main('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print(count_m)
print()

count_m = eso_instance.query_main(ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print(count_m)
print()

# - Query instrument - #

#count_m = eso_instance.query_instrument('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
#print(count_m)
#print()

In [ ]:
# BUG
os.environ["ESO_TAP_URL"] = "http://dfidev5.hq.eso.org:8123/tap_obs"
q_str = eso_instance.query_instrument('MUSE', count_only=True)
count_m = eso_instance.query_instrument(instrument='MUSE', ra=18, dec=0 , radius=1, count_only=True)
print(q_str, count_m)
del os.environ["ESO_TAP_URL"]

### Function Names - DECISION

 - Raw:    `query_main()` --> ?
 - Phase3: `query_surveys()` --> ? 
 - ISTs:   `query_instrument()` --> ?

## Missing items for this release

TAP (Done by 21th March):
 - ISTS --> Alberto
 - Apex quicklook products --> Alberto

Astroquery (Done by 1st April):
 - Agree on function names ----> ALL, EVERYONE
 - Bug fixing and improvements --> Juan
 - Docstrings to the public functions --> Juan
 - Authenticated queries --> Juan
 - Documentation --> Ashley, Juan

Done by 15h April
 - Internal review and testing (ESO) --> Ashley, Stefano, Alberto; Julien, Johan, Eric, Amelia, Anna, Antoine; Paranal (Andrea); Friendly external users.
 - External review --> astroquery people - Submit by 30th of April

## Upcoming features for next releases

To be discussed at the moment of submiting PR for review:
 
 - query_catalogues
 - SkyCoord
 - query_region
 - query_object
 - OR, >, <, between, like, ...
 - Uncorrelated random sample queries
 - cutouts - definition and download
 - asyncronous queries
 - ? previews, ssap, datalink
 - parameter `pint_query = True` prints the query executed. Example: `query_main(..., print_query=True)`
 - sort results (order by)

<hr style="border:2px solid #0281c9"> </hr>

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;"> </h1> 
  <h2 style="color: #0281c9; font-weight: bold;"> </h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

In [ ]:
from astroquery.eso import Eso
import os

eso_instance = Eso()
eso_instance.maxrec = -1
l1 = eso_instance.list_surveys()
l2 = eso_instance.query_tap_service("select distinct obs_survey from ivoa.ObsCore")

#del os.environ["ESO_TAP_URL"]
#eso_instance.tap_url()

In [ ]:
eso_instance.query_instrument('midi', column_filters={'target':'NGC 4151',
                                                     'exp_start':'2007-01-01',
                                                     'etime':'2008-01-01'},

                             columns=['night'])

In [ ]:
",".join(l1)

In [ ]:
l2 = list(l2["obs_survey"].data)


In [ ]:
set(l1) - set(l2)

In [ ]:
set(l2) - set(l1)